# Deadly Visualizations!!!

![Image](../images/viz_types_portada.png)

## Setup

First we need to create a basic setup which includes:

- Importing the libraries.

- Reading the dataset file (source [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259942408928&p=1259942408928&pagename=ProductosYServicios%2FPYSLayout)).

- Create a couple of columns and tables for the analysis.

__NOTE:__ some functions were already created in order to help you go through the challenge. However, feel free to perform any code you might need.

In [3]:
# imports

import sys
import re
sys.path.insert(0, "../modules")

import numpy as np
import pandas as pd

import plotly.express as px
import cufflinks as cf
cf.go_offline()

import module as mod     # functions are include in module.py

In [4]:
# read dataset

deaths = pd.read_csv('../data/7947.csv', sep=';', thousands='.')

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [5]:
# add some columns...you'll need them later

deaths['cause_code'] = deaths['Causa de muerte'].apply(mod.cause_code)
deaths['cause_group'] = deaths['Causa de muerte'].apply(mod.cause_types)
deaths['cause_name'] = deaths['Causa de muerte'].apply(mod.cause_name)

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


In [6]:
# lets check the categorical variables

var_list = ['Sexo', 'Edad', 'Periodo', 'cause_code', 'cause_name', 'cause_group']

categories = mod.cat_var(deaths, var_list)
categories

,categorical_variable,number_of_possible_values,values
0,cause_code,117,"[001-102, 001-008, 001, 002, 003, 004, 005, 00..."
1,cause_name,117,"[I-XXII.Todas las causas, I.Enfermedades infec..."
2,Periodo,39,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."
3,Edad,22,"[Todas las edades, Menos de 1 año, De 1 a 4 añ..."
4,Sexo,3,"[Total, Hombres, Mujeres]"
5,cause_group,2,"[Multiple causes, Single cause]"


In [7]:
# we need also to create a causes table for the analysis

causes_table = deaths[['cause_code', 'cause_name']].drop_duplicates().sort_values(by='cause_code').reset_index(drop=True)

causes_table

,cause_code,cause_name
0,001,Enfermedades infecciosas intestinales
1,001-008,I.Enfermedades infecciosas y parasitarias
2,001-102,I-XXII.Todas las causas
3,002,Tuberculosis y sus efectos tardíos
4,003,Enfermedad meningocócica
...,...,...
112,098,Suicidio y lesiones autoinfligidas
113,099,Agresiones (homicidio)
114,100,Eventos de intención no determinada
115,101,Complicaciones de la atención médica y quirúrgica


In [8]:
# And some space for free-style Pandas!!! (e.g.: df['column_name'].unique())

deaths['cause_name'].unique()






array(['I-XXII.Todas las causas',
       'I.Enfermedades infecciosas y parasitarias',
       'Enfermedades infecciosas intestinales',
       'Tuberculosis y sus efectos tardíos', 'Enfermedad meningocócica',
       'Septicemia', 'Hepatitis vírica', 'SIDA',
       'VIH+ (portador, evidencias de laboratorio del VIH, ...)',
       'Resto de enfermedades infecciosas y parasitarias y sus efectos tardíos',
       'II.Tumores',
       'Tumor maligno del labio, de la cavidad bucal y de la faringe',
       'Tumor maligno del esófago', 'Tumor maligno del estómago',
       'Tumor maligno del colon',
       'Tumor maligno del recto, de la porción rectosigmoide y del ano',
       'Tumor maligno del hígado y vías biliares intrahepáticas',
       'Tumor maligno del páncreas', 'Otros tumores malignos digestivos',
       'Tumor maligno de la laringe',
       'Tumor maligno de la tráquea, de los bronquios y del pulmón',
       'Otros tumores malignos respiratorios e intratorácicos',
       'Tumores malig

## Lets make some transformations

Eventhough the dataset is pretty clean, the information is completely denormalized as you could see. For that matter a collection of methods (functions) are available in order to generate the tables you might need:

- `row_filter(df, cat_var, cat_values)` => Filter rows by any value or group of values in a categorical variable.

- `nrow_filter(df, cat_var, cat_values)` => The same but backwards. 

- `groupby_sum(df, group_vars, agg_var='Total', sort_var='Total')` => Add deaths by a certain variable.

- `pivot_table(df, col, x_axis, value='Total')`=> Make some pivot tables, you might need them...

__NOTE:__ be aware that the filtering methods can perform a filter at a time. Feel free to perform the filter you need in any way you want or feel confortable with.

In [95]:
# Example 1

dataset = mod.row_filter(deaths, 'Sexo', ['Total'])
dataset = mod.row_filter(dataset, 'Edad', ['Todas las edades'])
dataset.head()


,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
0,001-102 I-XXII.Todas las causas,Total,Todas las edades,2018,427721,001-102,Multiple causes,I-XXII.Todas las causas
1,001-102 I-XXII.Todas las causas,Total,Todas las edades,2017,424523,001-102,Multiple causes,I-XXII.Todas las causas
2,001-102 I-XXII.Todas las causas,Total,Todas las edades,2015,422568,001-102,Multiple causes,I-XXII.Todas las causas
3,001-102 I-XXII.Todas las causas,Total,Todas las edades,2016,410611,001-102,Multiple causes,I-XXII.Todas las causas
4,001-102 I-XXII.Todas las causas,Total,Todas las edades,2012,402950,001-102,Multiple causes,I-XXII.Todas las causas


In [96]:
# Example 2

group = ['cause_code','Periodo']
dataset = mod.groupby_sum(deaths, group)
dataset.head()


,cause_code,Periodo,Total
0,001-102,2018,1710884
1,001-102,2017,1698092
2,001-102,2015,1690272
3,001-102,2016,1642444
4,001-102,2012,1611800


In [97]:
# Example 3

dataset = mod.pivot_table(dataset, 'cause_code', 'Periodo')
dataset.head()


cause_code,Periodo,001,001-008,001-102,002,003,004,005,006,007,...,093,094,095,096,097,098,099,100,101,102
0,1980,1620,15768,1157376,5904,2008,3448,436,0,0,...,4956,1432,184,692,16748,6608,1496,28,968,96
1,1981,1404,15124,1173544,6332,1656,3344,348,0,0,...,4700,1200,156,1396,17472,6872,1284,336,908,208
2,1982,1308,13488,1146620,5352,1240,3104,316,0,0,...,4864,956,200,1000,18616,7404,1228,440,1132,52
3,1983,1212,13100,1210276,5152,1072,3152,336,0,0,...,4788,1464,148,884,18392,8724,1560,1276,1500,56
4,1984,1228,12928,1197636,4564,964,3704,424,0,0,...,4716,1244,164,1020,14696,9972,1812,1144,1636,76


## ...and finally, show me some insights with Plotly!!!

In [65]:
# Cufflinks histogram
'''
dataset_column.iplot(kind='hist',
                     title='VIZ TITLE',
                     yTitle='AXIS TITLE',
                     xTitle='AXIS TITLE')
'''

"\ndataset_column.iplot(kind='hist',\n                     title='VIZ TITLE',\n                     yTitle='AXIS TITLE',\n                     xTitle='AXIS TITLE')\n"

In [105]:
dataset['006'].iplot(kind='hist',
                     title='VIZ TITLE',
                     yTitle='AXIS TITLE',
                     xTitle='AXIS TITLE')

In [ ]:
# Cufflinks bar plot
'''
dataset_bar.iplot(kind='bar',
                  x='VARIABLE',
                  xTitle='AXIS TITLE',
                  yTitle='AXIS TITLE',
                  title='VIZ TITLE')
'''

In [104]:
dataset.iplot(kind='bar',
                  x='Periodo',
                  xTitle='AXIS TITLE',
                  yTitle='AXIS TITLE',
                  title='VIZ TITLE')

In [ ]:
# Cufflinks line plot
'''
dataset_line.iplot(kind='line',
                   x='VARIABLE',
                   xTitle='AXIS TITLE',
                   yTitle='AXIS TITLE',
                   title='VIZ TITLE')
'''

In [48]:
dataset.iplot(kind='line',
                   x='Periodo',
                   xTitle='AXIS TITLE',
                   yTitle='AXIS TITLE',
                   title='VIZ TITLE')

In [ ]:
# Cufflinks scatter plot
'''
dataset_scatter.iplot(x='VARIABLE', 
                      y='VARIABLE', 
                      categories='VARIABLE',
                      xTitle='AXIS TITLE', 
                      yTitle='AXIS TITLE',
                      title='VIZ TITLE')
'''

In [55]:
dataset.iplot(x='013', 
                      y='010', 
                      categories='001',
                      xTitle='AXIS TITLE', 
                      yTitle='AXIS TITLE',
                      title='VIZ TITLE')

ValueError: 
    Invalid value of type 'numpy.int64' received for the 'name' property of scatter3d
        Received value: 405

    The 'name' property is a string and must be specified as:
      - A string
      - A number that will be converted to a string

# PROBAMOS

In [113]:
dataset = mod.row_filter(deaths,'cause_name',['Trastornos mentales debidos al uso de alcohol'])

In [156]:
dataset = dataset[dataset['Sexo']=='Total']
dataset = dataset[dataset['Edad']!='Todas las edades']
dataset

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
78,047 Trastornos mentales debidos al uso de alc...,Total,De 55 a 59 años,1989,63,047,Single cause,Trastornos mentales debidos al uso de alcohol
79,047 Trastornos mentales debidos al uso de alc...,Total,De 45 a 49 años,1983,62,047,Single cause,Trastornos mentales debidos al uso de alcohol
81,047 Trastornos mentales debidos al uso de alc...,Total,De 50 a 54 años,1983,58,047,Single cause,Trastornos mentales debidos al uso de alcohol
82,047 Trastornos mentales debidos al uso de alc...,Total,De 45 a 49 años,1981,56,047,Single cause,Trastornos mentales debidos al uso de alcohol
85,047 Trastornos mentales debidos al uso de alc...,Total,De 55 a 59 años,2009,55,047,Single cause,Trastornos mentales debidos al uso de alcohol
...,...,...,...,...,...,...,...,...
2568,047 Trastornos mentales debidos al uso de alc...,Total,De 1 a 4 años,2006,0,047,Single cause,Trastornos mentales debidos al uso de alcohol
2569,047 Trastornos mentales debidos al uso de alc...,Total,De 1 a 4 años,2005,0,047,Single cause,Trastornos mentales debidos al uso de alcohol
2570,047 Trastornos mentales debidos al uso de alc...,Total,De 1 a 4 años,2004,0,047,Single cause,Trastornos mentales debidos al uso de alcohol
2571,047 Trastornos mentales debidos al uso de alc...,Total,De 1 a 4 años,2003,0,047,Single cause,Trastornos mentales debidos al uso de alcohol


In [255]:
dataset2 = mod.row_filter(deaths,'cause_name',['Trastornos mentales debidos al uso de alcohol'])
dataset2 = dataset2[dataset2['Sexo']=='Total']
dataset2 = dataset2[dataset2['Edad']=='Todas las edades']
dataset2 = dataset2.sort_values(by= 'Periodo',ascending=False)
dataset2

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
70,047 Trastornos mentales debidos al uso de alc...,Total,Todas las edades,2018,230,047,Single cause,Trastornos mentales debidos al uso de alcohol
67,047 Trastornos mentales debidos al uso de alc...,Total,Todas las edades,2017,233,047,Single cause,Trastornos mentales debidos al uso de alcohol
61,047 Trastornos mentales debidos al uso de alc...,Total,Todas las edades,2016,249,047,Single cause,Trastornos mentales debidos al uso de alcohol
38,047 Trastornos mentales debidos al uso de alc...,Total,Todas las edades,2015,278,047,Single cause,Trastornos mentales debidos al uso de alcohol
50,047 Trastornos mentales debidos al uso de alc...,Total,Todas las edades,2014,260,047,Single cause,Trastornos mentales debidos al uso de alcohol
58,047 Trastornos mentales debidos al uso de alc...,Total,Todas las edades,2013,253,047,Single cause,Trastornos mentales debidos al uso de alcohol
18,047 Trastornos mentales debidos al uso de alc...,Total,Todas las edades,2012,298,047,Single cause,Trastornos mentales debidos al uso de alcohol
28,047 Trastornos mentales debidos al uso de alc...,Total,Todas las edades,2011,288,047,Single cause,Trastornos mentales debidos al uso de alcohol
32,047 Trastornos mentales debidos al uso de alc...,Total,Todas las edades,2010,286,047,Single cause,Trastornos mentales debidos al uso de alcohol
39,047 Trastornos mentales debidos al uso de alc...,Total,Todas las edades,2009,278,047,Single cause,Trastornos mentales debidos al uso de alcohol


In [248]:
dataset.iplot(kind='bar',
                  x='Periodo',
                  y='Total',
                  z ='Edad',
                  xTitle='Periodo',
                  yTitle='Número de muertes',
                  title='Trastornos mentales debido al alcohol')
dataset.iplot(x='Periodo', 
                      y='Total', 
                      categories='Edad',
                      xTitle='Periodo', 
                      yTitle='Número de muertes',
                      title='Fallecidos por alcohol')

In [210]:
px.bar(dataset,x='Periodo',y='Total',color='Edad')
#px.line(dataset,x='Total')

In [257]:
dataset2.iplot(kind='line',
                   y='Total',
                   x='Periodo',
                   
                   xTitle='Periodo',
                   yTitle='Muertes',
                   title='Muertes por alcohol')